In [ ]:
!pip install torchtext==0.4.0

In [ ]:
torch.__version__

'1.9.0+cu102'

In [ ]:
import torch
import torch.nn as nn
import torchvision.datasets as dset
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
from torch.autograd import Variable
import torchtext
import numpy as np
from data_loader import DataLoader

In [ ]:
# hyper-parameters
batch_size = 128
num_epochs = 10

word_vec_size = 256
dropout_p = 0.3

hidden_size = 512
num_layers = 4

learning_rate = 0.001 

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

### SMS train, test dataset 가져오기

In [ ]:
loaders = DataLoader(
    train_fn = './sms.maxlen.uniq.shuf.train.tsv',
    batch_size = batch_size,
    valid_ratio = .2,
    device = -1, 
    max_vocab = 999999,
    min_freq = 5,
)

In [ ]:
test_loaders = DataLoader(
    train_fn = './sms.maxlen.uniq.shuf.test.tsv',
    batch_size = batch_size,
    valid_ratio = .01,
    device = -1,
    max_vocab = 999999,
    min_freq = 5,
)

### 대략적인 데이터 형태

In [ ]:
print("|train| = ", len(loaders.train_loader.dataset),
      "|valid| = ", len(loaders.valid_loader.dataset))
vocab_size = len(loaders.text.vocab)
num_classes = len(loaders.label.vocab)
print("|vocab| = ", vocab_size, "|classes| = ", num_classes)

|train| =  3722 |valid| =  931
|vocab| =  1539 |classes| =  2


### 데이터 로드 함수

In [ ]:
n = 3 # 샘플로 출력할 데이터 개수
for i, data in enumerate(loaders.train_loader):
  labels = data.label
  texts = data.text

  if i > n:
    break
  print("[%d]" % i)
  print("한 번에 로드되는 데이터 크기 : ", len(labels))

  # 출력
  for j in range(n):
    label = labels[j].numpy() # tensor -> numpy로 변환
    text = texts[j].numpy()
    print("label : ", label)
    print("text : ", text.shape) # shape 을 떼면 raw 데이터가 나온다.

[0]
한 번에 로드되는 데이터 크기 :  128
label :  0
text :  (6,)
label :  0
text :  (6,)
label :  0
text :  (6,)
[1]
한 번에 로드되는 데이터 크기 :  128
label :  0
text :  (12,)
label :  1
text :  (12,)
label :  0
text :  (12,)
[2]
한 번에 로드되는 데이터 크기 :  128
label :  0
text :  (54,)
label :  0
text :  (54,)
label :  0
text :  (54,)
[3]
한 번에 로드되는 데이터 크기 :  128
label :  0
text :  (13,)
label :  0
text :  (13,)
label :  0
text :  (13,)


In [ ]:
class RNN(nn.Module):
  def __init__(self, 
               input_size, # vocab_size
               word_vec_size, # word embedding vector 차원
               hidden_size, # bidirectional LSTM 의 hidden state & cell state 의 size
               n_classes, 
               num_layers=4, # 쌓을 레이어 개수
               dropout_p=0.3):
    super(RNN, self).__init__()

    self.input_size = input_size
    self.word_vec_size = word_vec_size
    self.hidden_size = hidden_size
    self.n_classes = num_classes
    self.num_layers = num_layers
    self.dropout_p = dropout_p

    # 입력 차원(vocab_size), 출력 차원(word_vec_size)
    self.emb = nn.Embedding(input_size, word_vec_size)

    self.lstm = nn.LSTM(input_size=word_vec_size,
                        hidden_size=hidden_size,
                        num_layers=num_layers,
                        dropout=dropout_p,
                        batch_first=True,
                        bidirectional=True)
    self.fc = nn.Linear(hidden_size*2, num_classes)

    # LogSoftmax + NLLLoss instead of Softmax + CrossEntropy
    self.activation = nn.LogSoftmax(dim=-1) # 마지막 차원에 softmax 씌워줌

  def forward(self, x):
    # x : (batch_size, length)
    x = self.emb(x)

    # x : (batch_size, length, word_vec_size)
    x, _ = self.lstm(x) # x : output, _ : 마지막 time step의 hidden state & cell state

    # x : (batch_size, length, hidden_size * 2)
    # x[:, -1] : (batch_size, 1, hidden_size * 2)
    out = self.activation(self.fc(x[:, -1])) # 마지막 time step
    # self.fc(x[:, -1]) : (batch_size, num_classes)

    return out

In [ ]:
model = RNN(input_size=vocab_size,
            word_vec_size=word_vec_size,
            hidden_size=hidden_size,
            n_classes=num_classes,
            num_layers=num_layers,
            dropout_p=dropout_p)

In [ ]:
def ComputeAccr(dloader, imodel):
  correct = 0
  total = 0

  model.eval() # test mode
  for i, data in enumerate(dloader): # batch_size 만틈
    texts = data.text.to(device) # (batch_size, length)
    labels = data.label.to(device) # (batch_size, num_classes)

    # to(device) 를 붙일 경우, 내가 정한 device에서 실행되도록 한다.

    # Forward prop
    output = model(texts) # (batch_size, num_classes)
    _, output_index = torch.max(output, 1) # (batch_size, 1)

    total += labels.size(0)
    correct += (output_index == labels).sum().float()

  model.train()
  return (100 * correct / total).numpy()

In [ ]:
print("Accuracy of Test Data : %.2f" % ComputeAccr(loaders.valid_loader, model))

Accuracy of Test Data : 12.89


### loss, optimizer

In [ ]:
loss_func = nn.NLLLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

### 학습

In [ ]:
total_step = len(loaders.train_loader)
for epoch in range(num_epochs):
  for i, data in enumerate(loaders.train_loader):
    texts = data.text.to(device) # (batch_size, length)
    labels = data.label.to(device) # (batch_size, num_classes)

    print("[%d]" % i)

    # Forward prop
    outputs = model(texts)
    loss = loss_func(outputs, labels)

    # Backward prop & optimizer
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (i + 1) % 10 == 0:
      print("Epoch [{}/{}], Step [{}/{}], Loss : {:.4f}, Accr : {:.2f}"
      .format(epoch + 1, num_epochs, i + 1, total_step, loss.item(), ComputeAccr(loaders.valid_loader, model)))
    

[0]
[1]
[2]
[3]
[4]
[5]
[6]
[7]
[8]
[9]
Epoch [1/10], Step [10/30], Loss : 0.2624, Accr : 87.11
[10]
[11]
[12]
[13]
[14]
[15]
[16]
[17]
[18]
[19]
Epoch [1/10], Step [20/30], Loss : 0.2505, Accr : 87.33
[20]
[21]
[22]
[23]
[24]
[25]
[26]
[27]
[28]
[29]
Epoch [1/10], Step [30/30], Loss : 0.2593, Accr : 88.18
[0]
[1]
[2]
[3]
[4]
[5]
[6]
[7]
[8]
[9]
Epoch [2/10], Step [10/30], Loss : 0.4700, Accr : 88.51
[10]
[11]
[12]
[13]
[14]
[15]
[16]
[17]
[18]
[19]
Epoch [2/10], Step [20/30], Loss : 0.1899, Accr : 88.72
[20]
[21]
[22]
[23]
[24]
[25]
[26]
[27]
[28]
[29]
Epoch [2/10], Step [30/30], Loss : 0.0446, Accr : 90.01
[0]
[1]
[2]
[3]
[4]
[5]
[6]
[7]
[8]
[9]
Epoch [3/10], Step [10/30], Loss : 0.2616, Accr : 85.61
[10]
[11]
[12]
[13]
[14]
[15]
[16]
[17]
[18]
[19]
Epoch [3/10], Step [20/30], Loss : 0.1160, Accr : 94.95
[20]
[21]
[22]
[23]
[24]
[25]
[26]
[27]
[28]
[29]
Epoch [3/10], Step [30/30], Loss : 0.2476, Accr : 95.81
[0]
[1]
[2]
[3]
[4]
[5]
[6]
[7]
[8]
[9]
Epoch [4/10], Step [10/30], Loss : 0

### 테스트

In [ ]:
print("Accuracy of Valid Data : %.2f" % ComputeAccr(loaders.valid_loader, model))

Accuracy of Valid Data : f


### 학습된 파라미터 저장

In [ ]:
netname = "./nets/rnn_weight.pkl"
torch.save(model, netname,)

FileNotFoundError: ignored

### 학습된 파라미터 로드

In [ ]:
netname = './nets/rnn_weight.pkl'
model = torch.load(netname)
print("Accuracy of Valid Data : %.sf" % ComputeAccr(loaders.valid_loader, model))